### original QR:

$A^{(0)}=A$
for $k=1,2,...$

$\quad Q^{(k)}R^{(k)}=A^{(k-1)}$ (QR factorization of $A^{(k-1)}$) 

$\quad A^{(k)}=R^{(k)}Q^{(k)}$ (Recombine factors in reverse order) 

Under certain assumptions, $A^{(k)}$ converges to upper traingular form((Schur decomposition)).

When $A$ is symmetric, converges to diagonal form.

$A^{(k)}$ and $A^{(k-1)}$ are connected by orthogonal similarity transformation: $A^{(k)} = (Q^{(k)})^{T} A^{(k-1)} Q^{(k)}$

Pure QR time complexity:  space complexity: 

<br>


To make this QR practical:
- Reduce $A$ to upper Hessenberg or tridiagonal form before iteration rather than working with full matrices. 
- Use shifts to accelerate convergence: factor $A^{(k)} - \mu^{(k)}I$ rather than $A^{(k)}$
- When an off-diagonal entry becomes negligible, deflate the problem by breaking $A^{(k)}$ into pieces (smaller submatrices)


怎么看iterative methods的convergence？

In [5]:
import numpy as np
import time

def pure_qr(A, max_iter=1000, tol=1e-10):
    A_k = A.copy()
    n = A.shape[0]
    errors = []
    
    start_time = time.time()
    for k in range(max_iter):
        Q, R = np.linalg.qr(A_k)
        A_k = R @ Q
        
        # 检查收敛性 (对于对称矩阵，非对角元素应趋于0)
        off_diag = np.tril(A_k, -1)
        err = np.linalg.norm(off_diag)
        errors.append(err)
        
        if err < tol:
            break
            
    return np.diag(A_k).copy(), time.time() - start_time, errors

### Simultaneous Iteration

Extend power method to a set of vectors rather than a single vector

The QR algorithm is equivalent to simultaneous iteration with $\hat{Q}^{(0)}=I$ where we define $$

### Convergence of pure QR algorithm

**Theorem**: Let $A$ be symmetric, and its distinct eigenvalues $|\lambda_1| > ... >|\lambda_m| $ and all the leading principal submatrices of $Q=[q_1, ..., q_m]$ are nonsingular. Then we have $A^{(k)} \rightarrow diag(\lambda_1, ..., \lambda_m), \quad Q^{(k)} \rightarrow Q$

In [6]:
def simultaneous_iteration(A, max_iter=1000, tol=1e-10):
    n = A.shape[0]
    Q = np.eye(n)
    errors = []
    
    start_time = time.time()
    for k in range(max_iter):
        Z = A @ Q
        Q, R = np.linalg.qr(Z)
        
        T = Q.T @ A @ Q
        off_diag = np.tril(T, -1)
        err = np.linalg.norm(off_diag)
        errors.append(err)
        
        if err < tol:
            break
            
    return np.diag(Q.T @ A @ Q).copy(), time.time() - start_time, errors

### Shifts


In conclusion,
- Rayleigh quotient: cubic convergenc if converges
- Wilkinson shift: at least quadratic convergence

To find such shift, usually we use **Gerschgorin’ theorem**:

or any $ m \times m $ matrix $ A $, symmetric or nonsymmetric. Every eigenvalue of $ A $ lies in at least one of the $ m $ circular disks in the complex plane with centers $ a_{ii} $ and radii $ \sum_{j \neq i} |a_{ij}| $. Moreover, if $ n $ of these disks form a connected domain that is disjoint from the other $ m - n $ disks, then there are precisely $ n $ eigenvalues of $ A $ within this domain.


In [7]:
def shifted_qr_simple(A, max_iter=1000, tol=1e-10):
    H = A.copy()
    n = H.shape[0]
    eigenvalues = []
    
    start_time = time.time()
    
    m = n
    while m > 0:
        if m == 1:
            eigenvalues.append(H[0, 0])
            break
            
        for k in range(max_iter):
            # Check for deflation (checking sub-diagonal element)
            if abs(H[m-1, m-2]) < tol:
                eigenvalues.append(H[m-1, m-1])
                H = H[:m-1, :m-1]
                m -= 1
                break
            
            # Rayleigh quotient shift
            mu = H[m-1, m-1]
            
            Q, R = np.linalg.qr(H - mu * np.eye(m))
            H = R @ Q + mu * np.eye(m)
        else:
            eigenvalues.append(H[m-1, m-1])
            H = H[:m-1, :m-1]
            m -= 1

    return np.array(eigenvalues).copy(), time.time() - start_time

In [8]:
# profiling:
np.random.seed(42)
n = 20
A_rand = np.random.randn(n, n)
A_sym = A_rand + A_rand.T

# 运行各个算法
print("Running Pure QR...")
eig_pure, time_pure, err_pure = pure_qr(A_sym)

print("Running Simultaneous Iteration...")
eig_sim, time_sim, err_sim = simultaneous_iteration(A_sym)

print("Running Shifted QR...")
eig_shift, time_shift = shifted_qr_simple(A_sym)

print("Running Numpy (LAPACK)...")
t0 = time.time()
eig_numpy = np.linalg.eigvalsh(A_sym)
time_numpy = time.time() - t0

# 排序以便比较
eig_pure.sort()
eig_sim.sort()
eig_shift.sort()
eig_numpy.sort()

# 计算误差
err_val_pure = np.linalg.norm(eig_pure - eig_numpy)
err_val_sim = np.linalg.norm(eig_sim - eig_numpy)
err_val_shift = np.linalg.norm(eig_shift - eig_numpy)

# 打印结果表
print(f"{'Algorithm':<25} | {'Time (s)':<10} | {'Error (vs Numpy)':<15}")
print("-" * 60)
print(f"{'Pure QR':<25} | {time_pure:.5f}      | {err_val_pure:.2e}")
print(f"{'Simultaneous Iteration':<25} | {time_sim:.5f}      | {err_val_sim:.2e}")
print(f"{'Shifted QR (Deflation)':<25} | {time_shift:.5f}      | {err_val_shift:.2e}")
print(f"{'Numpy (Standard)':<25} | {time_numpy:.5f}      | 0.00")

Running Pure QR...
Running Simultaneous Iteration...
Running Shifted QR...
Running Numpy (LAPACK)...
Algorithm                 | Time (s)   | Error (vs Numpy)
------------------------------------------------------------
Pure QR                   | 0.03095      | 9.67e-14
Simultaneous Iteration    | 0.02730      | 1.28e-14
Shifted QR (Deflation)    | 0.00089      | 3.52e-14
Numpy (Standard)          | 0.00006      | 0.00


/var/folders/wl/tst5j8ss72n81nsz8b26y1f40000gn/T/ipykernel_63313/2046245558.py:12: RuntimeWarning: divide by zero encountered in matmul
  A_k = R @ Q
/var/folders/wl/tst5j8ss72n81nsz8b26y1f40000gn/T/ipykernel_63313/2046245558.py:12: RuntimeWarning: overflow encountered in matmul
  A_k = R @ Q
/var/folders/wl/tst5j8ss72n81nsz8b26y1f40000gn/T/ipykernel_63313/2046245558.py:12: RuntimeWarning: invalid value encountered in matmul
  A_k = R @ Q
/var/folders/wl/tst5j8ss72n81nsz8b26y1f40000gn/T/ipykernel_63313/1474459717.py:8: RuntimeWarning: divide by zero encountered in matmul
  Z = A @ Q
/var/folders/wl/tst5j8ss72n81nsz8b26y1f40000gn/T/ipykernel_63313/1474459717.py:8: RuntimeWarning: overflow encountered in matmul
  Z = A @ Q
/var/folders/wl/tst5j8ss72n81nsz8b26y1f40000gn/T/ipykernel_63313/1474459717.py:8: RuntimeWarning: invalid value encountered in matmul
  Z = A @ Q
/var/folders/wl/tst5j8ss72n81nsz8b26y1f40000gn/T/ipykernel_63313/1474459717.py:11: RuntimeWarning: divide by zero encounter

根据Gemini说的，现在numpy / Scipy: Python 中最常用的 `numpy.linalg.eig` (针对一般矩阵) 和 `numpy.linalg.eigh` (针对symmetrix/Hermissian matrices) 

底层实现: 它们底层调用的是 LAPACK (Linear Algebra PACKage) 库中的 `GEEV` 或 `SYEV` 例程。

netlib: http://www.netlib.org/lapack/
mirror: https://github.com/Reference-LAPACK/lapack

命名规则：LAPACK 函数名通常以精度开头。
`s` (Single float), `d` (Double float), `c` (Complex float), `z` (Complex double).

因此在 Python (通常是双精度) 中我们调用 dsyev 和 dgeev。

参数差异：直接调用 LAPACK 需要你更了解参数含义（如 compute_v / JOBZ），但这能让你完全控制计算过程（例如只算特征值不算特征向量以节省时间）

LAPACK 使用的是非常复杂的 Implicitly Shifted QR Algorithm (隐式位移QR算法)，也就是讲义中 Shifted QR 的更高级、更数值稳定的版本（如 Francis QR step）?

它还结合了分块算法 (Blocking) 来优化缓存性能。

In [9]:
# 调用的话
import numpy as np
from scipy.linalg import lapack

# 设置打印精度
np.set_printoptions(precision=4, suppress=True)

# --- 1. 调用 DSYEV (针对对称矩阵) ---
# 对应 Fortran 中的 DSYEV 例程
print("=== Calling LAPACK DSYEV (Symmetric) ===")
n = 4
A = np.random.randn(n, n)
A_sym = A + A.T # 构造对称矩阵

# 调用 dsyev
# a: 输入矩阵
# compute_v=1: 计算特征向量 (对应 JOBZ='V')
# lower=0: 使用上三角部分 (对应 UPLO='U')
w, v, info = lapack.dsyev(A_sym, compute_v=1, lower=0)

if info == 0:
    print("Eigenvalues (w):", w)
    # v 是特征向量矩阵 (列向量)
else:
    print("Computation failed!")

print("\n")

# --- 2. 调用 DGEEV (针对一般矩阵) ---
# 对应 Fortran 中的 DGEEV 例程
print("=== Calling LAPACK DGEEV (General) ===")
A_gen = np.random.randn(n, n) # 一般非对称矩阵

# 调用 dgeev
# compute_vl=0: 不计算左特征向量 (JOBVL='N')
# compute_vr=1: 计算右特征向量 (JOBVR='V')
# 返回值: (wr, wi, vl, vr, info)
# wr: 特征值实部, wi: 特征值虚部
wr, wi, vl, vr, info = lapack.dgeev(A_gen, compute_vl=0, compute_vr=1)

if info == 0:
    # 组合实部和虚部
    eigvals = wr + 1j * wi
    print("Eigenvalues (complex):", eigvals)
    # vr 包含右特征向量
else:
    print("Computation failed!")

=== Calling LAPACK DSYEV (Symmetric) ===
Eigenvalues (w): [-4.0955 -1.5196  0.9233  2.6749]


=== Calling LAPACK DGEEV (General) ===
Eigenvalues (complex): [ 2.5459+0.j     -1.0388+0.9142j -1.0388-0.9142j -0.166 +0.j    ]
